<a href="https://colab.research.google.com/github/keshavkmr48/Kaggle/blob/master/Kaggle_dsb_kids_game.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Setting up kaggle API to download competition dataset**

In [9]:
!pip install kaggle --upgrade
! mkdir .kaggle
import json
token = {"username":"keshavkmr48","key":"014d22aa9430a95daaf028ec3e3463a2"}
with open('/content/.kaggle/kaggle.json','w') as file:
  json.dump(token,file)
! cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
!kaggle config set -n path -v{/content}
!chmod 600 /root/.kaggle/kaggle.json
# download competition data
!kaggle competitions download -c data-science-bowl-2019

Requirement already up-to-date: kaggle in /usr/local/lib/python3.6/dist-packages (1.5.6)
- path is now set to: {/content}
 99% 393M/397M [00:02<00:00, 139MB/s]
100% 397M/397M [00:02<00:00, 147MB/s]
  0% 0.00/10.8k [00:00<?, ?B/s]
100% 10.8k/10.8k [00:00<00:00, 9.74MB/s]
 59% 24.0M/40.8M [00:00<00:00, 27.8MB/s]
100% 40.8M/40.8M [00:00<00:00, 100MB/s] 
  0% 0.00/262k [00:00<?, ?B/s]
100% 262k/262k [00:00<00:00, 81.2MB/s]
  0% 0.00/399k [00:00<?, ?B/s]
100% 399k/399k [00:00<00:00, 125MB/s]


In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import xgboost as xgb
from xgboost import XGBClassifier, XGBRegressor
from xgboost import plot_importance
# from catboost import CatBoostRegressor
from matplotlib import pyplot
# import shap
from time import time
from tqdm import tqdm_notebook as tqdm
from collections import Counter
from scipy import stats
import lightgbm as lgb
from sklearn.metrics import cohen_kappa_score, mean_squared_error
from sklearn.model_selection import KFold, StratifiedKFold
import gc, os
import json
pd.set_option('display.max_columns', 1000)


In [12]:
# list data in the downloaded folder
os.listdir('{/content}/competitions/data-science-bowl-2019')
os.chdir('{/content}/competitions/data-science-bowl-2019')
! unzip test.csv.zip
! unzip train_labels.csv.zip
! unzip train.csv.zip
! rm -rf *.zip
os.listdir()

Archive:  test.csv.zip
  inflating: test.csv                
Archive:  train_labels.csv.zip
  inflating: train_labels.csv        
Archive:  train.csv.zip
  inflating: train.csv               


['train.csv',
 'train_labels.csv',
 'specs.csv',
 'test.csv',
 'sample_submission.csv']

In [0]:
def read_data():
    print('Reading train.csv file....')
    train = pd.read_csv('train.csv')
    print('Training.csv file have {} rows and {} columns'.format(train.shape[0], train.shape[1]))

    print('Reading test.csv file....')
    test = pd.read_csv('test.csv')
    print('Test.csv file have {} rows and {} columns'.format(test.shape[0], test.shape[1]))

    print('Reading train_labels.csv file....')
    train_labels = pd.read_csv('train_labels.csv')
    print('Train_labels.csv file have {} rows and {} columns'.format(train_labels.shape[0], train_labels.shape[1]))

    print('Reading specs.csv file....')
    specs = pd.read_csv('specs.csv')
    print('Specs.csv file have {} rows and {} columns'.format(specs.shape[0], specs.shape[1]))

    print('Reading sample_submission.csv file....')
    sample_submission = pd.read_csv('sample_submission.csv')
    print('Sample_submission.csv file have {} rows and {} columns'.format(sample_submission.shape[0], sample_submission.shape[1]))
    return train, test, train_labels, specs, sample_submission

In [0]:
train, test, train_labels, specs, sample_submission = read_data()

Reading train.csv file....


In [0]:
temp_train = pd.DataFrame(list(train.event_data.apply(lambda x : json.loads(x))))